# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

# Solution:





# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

In [5]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


## Answer #1

In [10]:
s = 0
limit = 5
generator = square_root_generator(limit)
for sqrt_value in generator:
    s += sqrt_value
   
print(s)

8.382332347441762


## Answer #2

In [11]:
limit = 13
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [12]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


## Answer #3

### Creating the pipeline and loading data with the first generator

In [14]:
import dlt
import duckdb


# Define the pipeline
pipeline = dlt.pipeline(
    pipeline_name= 'my_pipeline',
    destination='duckdb',
    dataset_name='mydata'
)

# Run the pipeline with the generator above.
load_info = pipeline.run(people_1(), 
                         table_name="users",
                         write_disposition="replace",
                    primary_key="ID"
                         )


### Display the tables

In [15]:
conn = duckdb.connect(f"{pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ users               │
└─────────────────────┘

### Sum of Age for the first generator

In [16]:
sum_age = conn.sql("SELECT SUM(Age) FROM users").df()
display(sum_age)

,sum(Age)
0,140.0


### Now we'll append the second generator to the first generator

In [17]:
# Run the pipeline with the generator above.
load_info = pipeline.run(people_2(), 
                         table_name="users",
                         write_disposition="append"
                         )

### Sum resulting of the Append

In [18]:
sum_age = conn.sql("SELECT SUM(Age) FROM users").df()
display(sum_age)

,sum(Age)
0,353.0


# 3. Merge a generator

# Question #4
Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


In [20]:
# Run the pipeline with the generator above.
load_info = pipeline.run(people_1(), 
                         table_name="users_merge",
                         write_disposition="replace",
                    primary_key="ID"
                         )

load_info = pipeline.run(people_2(), 
                         table_name="users_merge",
                         write_disposition="merge",
                    primary_key="ID"
                         )

print( "Display the content of the table")

display(conn.sql("SELECT * FROM users_merge").df())


sum_age = conn.sql("SELECT SUM(Age) FROM users_merge").df()
display(sum_age)

Display the content of the table


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708033745.609778,Qwk6GpY+NHjxFw,None
1,2,Person_2,27,City_A,1708033745.609778,qKMSMPm0cwF9UQ,None
2,8,Person_8,38,City_B,1708033746.1164339,xWJzlZGmssQS2A,Job_8
3,5,Person_5,35,City_B,1708033746.1164339,8zEtX0XYwvt93g,Job_5
4,3,Person_3,33,City_B,1708033746.1164339,g/cAQFxImDkX0Q,Job_3
5,6,Person_6,36,City_B,1708033746.1164339,nQCqCoDOdwAPAg,Job_6
6,4,Person_4,34,City_B,1708033746.1164339,VQrkh4BnHonNXg,Job_4
7,7,Person_7,37,City_B,1708033746.1164339,uMycycP6H8KnCA,Job_7


,sum(Age)
0,266.0
